### Import Packages

In [44]:
import pandas as pd

In [45]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

### Load All Datasets And Check First Couple Of Rows

In [28]:
steam200kinitial = pd.read_csv('C:/Users/Joshua/Documents/git/EntityFinalProject-main/steam-200k.csv')
gamesales2017initial = pd.read_csv('C:/Users/Joshua/Documents/git/EntityFinalProject-main/Video_Game_Sales_as_of_Jan_2017.csv')

In [29]:
steam200kinitial.head(5)

,151603712,The Elder Scrolls V Skyrim,purchase,1.0,0
0,151603712,The Elder Scrolls V Skyrim,play,273.0,0
1,151603712,Fallout 4,purchase,1.0,0
2,151603712,Fallout 4,play,87.0,0
3,151603712,Spore,purchase,1.0,0
4,151603712,Spore,play,14.9,0


In [30]:
gamesales2017initial.head(5)

,Name,Genre
0,The Sims 3,Simulation
1,World of Warcraft,Role-Playing
2,Diablo III,Role-Playing
3,Microsoft Flight Simulator,Simulation
4,StarCraft II: Wings of Liberty,Strategy


In [31]:
steam200k = steam200kinitial.rename(columns={'151603712': 'User', 'The Elder Scrolls V Skyrim': 'Name', 'purchase':'Behavior', '1.0':'Playtime'})
steam200k.drop('0', axis=1, inplace=True)
steam200k.head(5)

,User,Name,Behavior,Playtime
0,151603712,The Elder Scrolls V Skyrim,play,273.0
1,151603712,Fallout 4,purchase,1.0
2,151603712,Fallout 4,play,87.0
3,151603712,Spore,purchase,1.0
4,151603712,Spore,play,14.9


In [32]:
steam200k.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 199999 entries, 0 to 199998
Data columns (total 4 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   User      199999 non-null  int64  
 1   Name      199999 non-null  object 
 2   Behavior  199999 non-null  object 
 3   Playtime  199999 non-null  float64
dtypes: float64(1), int64(1), object(2)
memory usage: 6.1+ MB


In [33]:
steam200k.nunique()

User        12393
Name         5155
Behavior        2
Playtime     1593
dtype: int64

### Descriptive Statitics

In [34]:
steam200k.describe()

,User,Playtime
count,1.999990e+05,199999.000000
mean,1.036556e+08,17.874468
std,7.208084e+07,138.057292
min,5.250000e+03,0.100000
25%,4.738420e+07,1.000000
50%,8.691201e+07,1.000000
75%,1.542309e+08,1.300000
max,3.099031e+08,11754.000000


In [35]:
#join datasets
steam200k_genre = steam200k.merge(gamesales2017initial, how='left', on='Name')
steam200k_genre.head(5)

,User,Name,Behavior,Playtime,Genre
0,151603712,The Elder Scrolls V Skyrim,play,273.0,NaN
1,151603712,Fallout 4,purchase,1.0,Role-Playing
2,151603712,Fallout 4,play,87.0,Role-Playing
3,151603712,Spore,purchase,1.0,Strategy
4,151603712,Spore,play,14.9,Strategy


In [36]:
#remove rows where genre is NaN
steam200k_genreR=steam200k_genre.dropna()
steam200k_genreR.head(5)

,User,Name,Behavior,Playtime,Genre
1,151603712,Fallout 4,purchase,1.0,Role-Playing
2,151603712,Fallout 4,play,87.0,Role-Playing
3,151603712,Spore,purchase,1.0,Strategy
4,151603712,Spore,play,14.9,Strategy
7,151603712,Left 4 Dead 2,purchase,1.0,Shooter


In [37]:
#check if there are any users who didnt purchase
#tempdf = steam200k_genreR
#tempdf.drop(tempdf[tempdf.Behavior == 'play'].index, inplace=True)
#tempdf['Playtime'].value_counts()

#showed every entry was a 1.0 but it started to bug my code when I left it uncommented

In [38]:
#drop rows that have behavior of purchase because every user has purchased the game
steam200k_genreR.drop(steam200k_genreR[steam200k_genreR.Behavior == 'purchase'].index, inplace=True)

#Add new column and set 1 because every user has purchased the game
steam200k_genreR['Purchace'] = 1

#Add new column and set 0 by default
steam200k_genreR['Play'] = 0

steam200k_genreR.head(5)

C:\Users\Joshua\anaconda3\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
C:\Users\Joshua\AppData\Local\Temp/ipykernel_20360/2637236718.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  steam200k_genreR['Purchace'] = 1
C:\Users\Joshua\AppData\Local\Temp/ipykernel_20360/2637236718.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pan

,User,Name,Behavior,Playtime,Genre,Purchace,Play
2,151603712,Fallout 4,play,87.0,Role-Playing,1,0
4,151603712,Spore,play,14.9,Strategy,1,0
8,151603712,Left 4 Dead 2,play,8.9,Shooter,1,0
16,151603712,Left 4 Dead,play,3.3,Shooter,1,0
20,151603712,Tomb Raider,play,2.5,Action,1,0


In [39]:
#set 1 or 0 if game was played
steam200k_genreR.loc[steam200k_genreR['Playtime'] > .1, 'Play'] = 1

#drop behavior column
steam200k_genreR.drop('Behavior', axis=1, inplace=True)

steam200k_genreR.head(5)

C:\Users\Joshua\anaconda3\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
C:\Users\Joshua\anaconda3\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,User,Name,Playtime,Genre,Purchace,Play
2,151603712,Fallout 4,87.0,Role-Playing,1,1
4,151603712,Spore,14.9,Strategy,1,1
8,151603712,Left 4 Dead 2,8.9,Shooter,1,1
16,151603712,Left 4 Dead,3.3,Shooter,1,1
20,151603712,Tomb Raider,2.5,Action,1,1


In [40]:
#export data wrangeled df to file
steam200k_genreR.to_csv(r'C:\Users\Joshua\Documents\git\EntityFinalProject-main\steam200k_genreR.csv', index = False)

In [46]:
sample_tf = TfidfVectorizer(analyzer='word', ngram_range=(1,1), min_df=0, stop_words='english')

In [49]:
sent_df = pd.DataFrame([['This is an important sentence.'], ['Here is another sentence.'], ['Important news arrived,'], ['Sentence incoming.']])
sent_df

,0
0,This is an important sentence.
1,Here is another sentence.
2,"Important news arrived,"
3,Sentence incoming.


In [51]:
sample_tfidf = sample_tf.fit_transform(sent_df[0])

In [52]:
sample_tfidf.shape

(4, 5)

In [53]:
sample_tfidf.todense()

matrix([[0.        , 0.77722116, 0.        , 0.        , 0.62922751],
        [0.        , 0.        , 0.        , 0.        , 1.        ],
        [0.61761437, 0.48693426, 0.        , 0.61761437, 0.        ],
        [0.        , 0.        , 0.84292635, 0.        , 0.53802897]])

In [54]:
print(sorted(sample_tf.vocabulary_.items(), key=lambda item: item[1]))

[('arrived', 0), ('important', 1), ('incoming', 2), ('news', 3), ('sentence', 4)]


### Full tf-idf

In [58]:
tf = TfidfVectorizer(analyzer='word', ngram_range=(1, 3), min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(steam200k_genreR['Name'])

In [59]:
tfidf_matrix.shape

(11931, 639)

### Create similarity matrix and rank items

In [61]:
cosine_similarities = cosine_similarity(tfidf_matrix) 
results = {}
for idx, row in steam200k_genreR.iterrows():
   similar_indices = cosine_similarities[idx].argsort()[:-100:-1] 
   similar_items = [(cosine_similarities[idx][i], steam200k_genreR['Name'][i]) for i in similar_indices] 
   results[row['Name']] = similar_items[1:]

KeyError: 6985